# Modul 3 - Demo

### Dataset : https://drive.google.com/file/d/1x2Xpa5T-ifi1xn-MtKA8HOV7E9Rk0NlJ/view?pli=1

Gunakan dataset di atas untuk melakukan serangkaian proses di bawah:

1. Handling Categorical Values:
- Identifikasi kolom-kolom kategorikal dalam dataset.
- Terapkan metode handling categorical value yang cocok pada 3 kolom yang ada dalam dataset (tidak boleh ketiganya metode yang sama, bisa proporsi 2 dan 1).
- Lakukan Binning pada kolom age dengan membaginya ke dalam 4 kelompok: "Muda", "Dewasa", "Paruh Baya", dan "Lanjut Usia".

2. Data Normalization:
- Gunakan Min-Max Scaling pada kolom balance.
- Gunakan Z-Score Scaling pada kolom duration.
- Gunakan Decimal Scaling pada kolom campaign.

3. Dimensionality Reduction:
- Lakukan Feature Selection dengan memilih hanya fitur yang memiliki korelasi tinggi terhadap variabel target (y). Gunakan korelasi Pearson atau metode seleksi fitur lain yang sesuai.
- Lakukan Feature Extraction menggunakan PCA (Principal Component Analysis) untuk mereduksi dimensi dataset menjadi hanya 5 fitur utama.

4. Data Splitting:
- Bagi dataset menjadi Train (70%), Validation (15%), dan Test (15%).
- Pastikan bahwa distribusi kelas dalam variabel target (y) tetap seimbang dalam proses pembagian data.

### Persiapan Awal

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

# Memuat dataset ke DataFrame
data = pd.read_csv('tugas3_genap.csv')  # Ganti 'dataset.csv' dengan path file Anda jika diperlukan
# Jika Anda langsung copy-paste data, gunakan ini:
# data = pd.read_csv(io.StringIO(<paste dataset Anda di sini>), delimiter=',')

# Ganti 'deposit' menjadi 'y' sebagai variabel target untuk konsistensi
data = data.rename(columns={'deposit': 'y'})

### 1. Handling Categorical Values

In [6]:
# Bagian 1: Handling Categorical Values

# Identifikasi kolom kategorikal
categorical_cols = data.select_dtypes(include=['object']).columns
print("Kolom kategorikal:", categorical_cols)

# Terapkan metode handling categorical value yang berbeda pada 3 kolom
# 1. One-Hot Encoding untuk 'job' (metode 1)
data_encoded = pd.get_dummies(data, columns=['job'], prefix='job')

# 2. Label Encoding untuk 'marital' (metode 2)
marital_mapping = {'single': 0, 'married': 1, 'divorced': 2}
data_encoded['marital'] = data_encoded['marital'].map(marital_mapping)

# 3. Frequency Encoding untuk 'education' (metode 3)
education_freq = data_encoded['education'].value_counts(normalize=True)
data_encoded['education'] = data_encoded['education'].map(education_freq)

# Binning kolom 'age' menjadi 4 kelompok
bins = [0, 25, 40, 60, np.inf]  # Rentang usia
labels = ['Muda', 'Dewasa', 'Paruh Baya', 'Lanjut Usia']
data_encoded['age_group'] = pd.cut(data_encoded['age'], bins=bins, labels=labels, include_lowest=True)

# Tampilkan hasil awal
print("Hasil setelah handling categorical values:")
print(data_encoded.head())

Kolom kategorikal: Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')
Hasil setelah handling categorical values:
   age  marital  education default  balance housing loan  contact  day month  \
0   59        1   0.490593      no     2343     yes   no  unknown    5   may   
1   56        1   0.490593      no       45      no   no  unknown    5   may   
2   41        1   0.490593      no     1270     yes   no  unknown    5   may   
3   55        1   0.490593      no     2476     yes   no  unknown    5   may   
4   54        1   0.330496      no      184      no   no  unknown    5   may   

   ...  job_housemaid  job_management  job_retired  job_self-employed  \
0  ...          False           False        False              False   
1  ...          False           False        False              False   
2  ...          False           False        False              False   
3  ...          False         

### 2. Data Normalization

In [7]:
# Bagian 2: Data Normalization

# Min-Max Scaling pada 'balance'
minmax_scaler = MinMaxScaler()
data_encoded['balance_scaled'] = minmax_scaler.fit_transform(data_encoded[['balance']])

# Z-Score Scaling pada 'duration'
zscore_scaler = StandardScaler()
data_encoded['duration_scaled'] = zscore_scaler.fit_transform(data_encoded[['duration']])

# Decimal Scaling pada 'campaign'
# Decimal scaling: bagi dengan 10^p di mana p adalah jumlah digit maksimum
max_campaign = data_encoded['campaign'].max()
p = len(str(int(max_campaign)))
data_encoded['campaign_scaled'] = data_encoded['campaign'] / (10 ** p)

# Tampilkan hasil normalisasi
print("Hasil setelah normalisasi:")
print(data_encoded[['balance', 'balance_scaled', 'duration', 'duration_scaled', 'campaign', 'campaign_scaled']].head())

Hasil setelah normalisasi:
   balance  balance_scaled  duration  duration_scaled  campaign  \
0     2343        0.104371      1042         1.930226         1   
1       45        0.078273      1467         3.154612         1   
2     1270        0.092185      1389         2.929901         1   
3     2476        0.105882       579         0.596366         1   
4      184        0.079851       673         0.867171         2   

   campaign_scaled  
0             0.01  
1             0.01  
2             0.01  
3             0.01  
4             0.02  


### 3. Dimensionality Reduction

In [11]:
# Bagian 3: Dimensionality Reduction

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif

# Misalkan data_encoded sudah ada dari langkah sebelumnya
# Pisahkan fitur dan target
X = data_encoded.drop(columns=['y', 'age'])  # Hilangkan 'age' karena sudah dibinning
y = data_encoded['y'].map({'yes': 1, 'no': 0})  # Encode target menjadi numerik

# Feature Selection menggunakan SelectKBest dengan f_classif (ANOVA F-value)
selector = SelectKBest(score_func=f_classif, k=10)  # Pilih 10 fitur terbaik
X_selected = selector.fit_transform(X.select_dtypes(include=[np.number]), y)

# Dapatkan nama fitur yang terpilih
selected_features = X.select_dtypes(include=[np.number]).columns[selector.get_support()].tolist()
print("Fitur terpilih:", selected_features)

# Feature Extraction menggunakan PCA
pca = PCA(n_components=5)  # Reduksi menjadi 5 komponen utama
X_pca = pca.fit_transform(X.select_dtypes(include=[np.number]))

# Tampilkan variansi yang dijelaskan oleh PCA
print("\nHasil PCA:")
print("Explained variance ratio untuk setiap komponen:", pca.explained_variance_ratio_)
print("Total explained variance:", sum(pca.explained_variance_ratio_))

# Tampilkan komponen PCA (loading factors)
print("\nKomponen PCA (Loading Factors):")
pca_components_df = pd.DataFrame(pca.components_, columns=X.select_dtypes(include=[np.number]).columns, index=[f'PC{i+1}' for i in range(5)])
print(pca_components_df)

# Buat DataFrame baru dengan fitur PCA
X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(5)])
print("\n5 Baris pertama dari dataset setelah PCA:")
print(X_pca_df.head())

Fitur terpilih: ['marital', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'balance_scaled', 'duration_scaled', 'campaign_scaled']

Hasil PCA:
Explained variance ratio untuk setiap komponen: [9.87438285e-01 1.14323578e-02 1.12156802e-03 6.69924415e-06
 6.81549761e-07]
Total explained variance: 0.9999995913569644

Komponen PCA (Loading Factors):
          marital  education   balance       day  duration  campaign  \
PC1  4.142814e-07  -0.000002  0.999997  0.000027  0.002443 -0.000012   
PC2 -1.214208e-05   0.000001 -0.002437 -0.000450  0.999946 -0.000321   
PC3 -1.811054e-04   0.000040 -0.000611 -0.006082  0.009653 -0.002603   
PC4  1.093810e-04  -0.000090 -0.000032  0.998886  0.000523  0.046412   
PC5  6.765958e-03  -0.001036  0.000011 -0.046372  0.000323  0.998803   

        pdays  previous  balance_scaled  duration_scaled  campaign_scaled  
PC1  0.000588  0.000022    1.135702e-05     7.037641e-06    -1.173648e-07  
PC2 -0.009657 -0.000191   -2.767789e-08     2.880752

### 4. Data Splitting

In [12]:
# Bagian 4: Data Splitting

# Gabungkan fitur PCA dengan target
data_final = pd.concat([X_pca_df, y.reset_index(drop=True)], axis=1)

# Bagi dataset menjadi Train (70%), Validation (15%), dan Test (15%) dengan stratifikasi
train_data, temp_data = train_test_split(data_final, test_size=0.3, stratify=data_final['y'], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['y'], random_state=42)

# Verifikasi distribusi kelas
print("Distribusi kelas di Train:")
print(train_data['y'].value_counts(normalize=True))
print("Distribusi kelas di Validation:")
print(val_data['y'].value_counts(normalize=True))
print("Distribusi kelas di Test:")
print(test_data['y'].value_counts(normalize=True))

# Tampilkan ukuran dataset
print(f"Ukuran Train: {train_data.shape}")
print(f"Ukuran Validation: {val_data.shape}")
print(f"Ukuran Test: {test_data.shape}")

Distribusi kelas di Train:
y
0    0.526174
1    0.473826
Name: proportion, dtype: float64
Distribusi kelas di Validation:
y
0    0.526284
1    0.473716
Name: proportion, dtype: float64
Distribusi kelas di Test:
y
0    0.52597
1    0.47403
Name: proportion, dtype: float64
Ukuran Train: (7813, 6)
Ukuran Validation: (1674, 6)
Ukuran Test: (1675, 6)


In [10]:
X_pca_df

,PC1,PC2,PC3,PC4,PC5
0,816.064611,668.501506,-46.293989,-10.713748,-0.909829
1,-1480.889912,1099.082493,-40.787008,-10.418853,-0.797001
2,-256.084324,1018.100647,-42.288580,-10.498327,-0.809124
3,947.933139,205.198498,-50.844726,-10.960195,-1.058077
4,-1343.830006,304.779643,-48.539249,-10.792243,-0.053115
...,...,...,...,...,...
11157,-1527.845243,-110.761807,-52.531625,3.932653,-1.890596
11158,-796.272759,-286.536977,-54.642277,-0.037705,1.249574
11159,-1500.092102,-211.825308,-53.520222,2.926423,-0.878040
11160,-1529.349664,-360.414605,118.176832,-7.084302,0.021699
